Adicionando os Pacotes que seram usandos e os arquivos

In [218]:
import pandas as pd
import os
from tabulate import tabulate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sqlite3


# Definir o caminho do arquivo CSV
labevents_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\LABEVENTS.csv'
labitems_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\D_LABITEMS.csv'
admissions_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\ADMISSIONS.csv'
patients_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\PATIENTS.csv'
# nome_diagnoses_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\D_ICD_DIAGNOSES.csv'
# diagnoses_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\DIAGNOSES_ICD.csv'
# drgcodes_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\DRGCODES.csv'
# microbiologyevents_path = r'C:\Users\ricak\OneDrive\Área de Trabalho\Projeto deyvid\mimic-iii-clinical-database-demo-1.4\MICROBIOLOGYEVENTS.csv'


Carregando os arquivos em chunks

In [219]:
# Definir o tamanho dos chunks
chunk_size = 1000

# Lista para armazenar os dados filtrados
dados_filtrados_labevents = []
dados_filtrados_labitems = []
dados_filtrados_admissions = []
dados_filtrados_patients = []
# dados_filtrados_nome_diagnoses = []
# dados_filtrados_diagnoses = []
# dados_filtrados_drgcodes = []
# dados_filtrados_microbiologyevents = []

# Ler LABEVENTS em chunks e processar cada chunk
for labevents_chunk in pd.read_csv(labevents_path, chunksize=chunk_size):
   
     # Adicionar o chunk filtrado à lista
    dados_filtrados_labevents.append(labevents_chunk)

# Ler D_LABITEMS em chunks e processar cada chunk
for labitems_chunk in pd.read_csv(labitems_path, chunksize=chunk_size):
    
        dados_filtrados_labitems.append(labitems_chunk)

# Ler ADMISSIONS em chunks e processar cada chunk
for admissions_chunk in pd.read_csv(admissions_path, chunksize=chunk_size):
    
        dados_filtrados_admissions.append(admissions_chunk)

# Ler PATIENTS em chunks e processar cada chunk
for patients_chunk in pd.read_csv(patients_path, chunksize=chunk_size):
    
        dados_filtrados_patients.append(patients_chunk)

# Ler NOME_DIAGNOSES em chunks e processar cada chunk
# for nome_diagnoses_chunk in pd.read_csv(nome_diagnoses_path, chunksize=chunk_size):
    
#         dados_filtrados_nome_diagnoses.append(nome_diagnoses_chunk)

# # Ler DIAGNOSES em chunks e processar cada chunk
# for diagnoses_chunk in pd.read_csv(diagnoses_path, chunksize=chunk_size):
    
#         dados_filtrados_diagnoses.append(diagnoses_chunk)

Retirando as colunas das tabelas usadas

In [220]:
df_filtrado_labevents = pd.concat(dados_filtrados_labevents, ignore_index=True)

df_filtrado_labitems = pd.concat(dados_filtrados_labitems, ignore_index=True)

df_filtrado_admissions = pd.concat(dados_filtrados_admissions, ignore_index=True)

df_filtrado_patients = pd.concat(dados_filtrados_patients, ignore_index=True)

# df_filtrado_2 = pd.concat(dados_filtrados_nome_diagnoses, ignore_index=True)

# df_filtrado_4 = pd.concat(dados_filtrados_diagnoses, ignore_index=True)

 #retirar a coluna 'row_id'

df_filtrado_labevents = df_filtrado_labevents.drop(columns=['row_id', 'charttime','hadm_id','value'])

df_filtrado_labitems = df_filtrado_labitems.drop(columns=['row_id','loinc_code'])

df_filtrado_admissions = df_filtrado_admissions.drop(columns=['row_id', 'admittime','dischtime','insurance','hadm_id','deathtime','admission_type','admission_location','discharge_location','edregtime','edouttime','has_chartevents_data'])

df_filtrado_patients = df_filtrado_patients.drop(columns=['row_id','expire_flag','dob','dod','dod_hosp','dod_ssn'])

# df_filtrado_2 = df_filtrado_2.drop(columns=['row_id','long_title'])

# df_filtrado_4 = df_filtrado_4.drop(columns=['row_id','seq_num','hadm_id'])

Vendo quais são as variáveis que são não numéricas na coluna value

In [221]:
# import pandas as pd

# # Supondo que seu DataFrame se chame 'df' e possua as colunas 'itemid' e 'value'
# # Carregar o DataFrame (ajuste para o seu caminho e arquivo)
# # df = pd.read_csv('seu_arquivo.csv')

# # Defina o itemid que você deseja investigar
# itemid_interesse = 51137

# # Filtrar o DataFrame para incluir apenas as linhas com o itemid desejado
# df_filtrado = Tabela_final[Tabela_final['itemid'] == itemid_interesse]

# # Filtrar apenas os valores não numéricos na coluna 'value'
# # Usamos a função `str.isnumeric` para identificar valores que não sejam números
# # `~` é o operador de negação para inverter a condição
# valores_nao_numericos = df_filtrado[~df_filtrado['value'].str.isnumeric()]

# # Ver as variáveis categóricas distintas presentes na coluna 'value' para o itemid desejado
# variaveis_categoricas = valores_nao_numericos['value'].unique()

# # Exibir o resultado
# print(f"Valores categóricos para o itemid {itemid_interesse}:")
# print(variaveis_categoricas)


Fazendo a Normalização dos dados por meio de min e max

In [222]:
# Inicializar o MinMaxScaler
scaler = MinMaxScaler()

# Verificar se a coluna 'valuenum' tem valores numéricos e não nulos antes da normalização
if 'valuenum' in df_filtrado_labevents:
    # Substituir valores NaN por zero ou outro valor de sua escolha, se necessário:
    df_filtrado_labevents['valuenum'] = df_filtrado_labevents['valuenum'].fillna(0)
    
    # Aplicar o scaler e transformar os valores para a escala entre 0 e 1
    df_filtrado_labevents['valuenum_normalized'] = scaler.fit_transform(df_filtrado_labevents[['valuenum']])


Juntando as tabelas em uma única

In [223]:
#Fazer a junção das tabelas

Junto_1 = pd.merge(df_filtrado_labevents, df_filtrado_labitems, on='itemid', how='inner')

Junto_2 = pd.merge(df_filtrado_admissions, df_filtrado_patients, on='subject_id', how='inner')

Tabela_final = pd.merge(Junto_1, Junto_2, on='subject_id', how='inner')

# Junto_4 = pd.merge(Junto_3, df_filtrado_4, on='subject_id', how='inner')

# Tabela_final = pd.merge(Junto_4, df_filtrado_2, on='icd9_code', how='inner')

from prettytable import PrettyTable

# Criar a tabela PrettyTable
table = PrettyTable()

# Adicionar os nomes das colunas
table.field_names = Tabela_final.columns

# Adicionar as linhas à tabela
for row in Tabela_final.itertuples(index=False):
    table.add_row(row)

# # # Retorna um DataFrame com os valores únicos da coluna
# valores_unicos_df = Tabela_final['valueuom'].drop_duplicates()
# valores_unicos_df.to_csv('valores_unicos.csv', index=False)

# print(valores_unicos_df)

# SELECT itemid, label, valueuom FROM Tabela_final WHERE itemid = 50868


# valores_unicos_df = pd.DataFrame(table.rows, columns=table.field_names)
# valores_unicos_df.to_csv('valores_unicos.csv', index=False)

# pd.set_option('display.max_rows', None)  # None para mostrar todas as linhas
# pd.set_option('display.max_columns', None)  # None para mostrar todas as colunas

# # Retorna um DataFrame com os valores únicos da coluna
# valores_unicos_df = Tabela_final['valuenum'].value_counts()
# print(valores_unicos_df)


Olhando quais valores estão ausentes na tabela e as diferentes unidades de medida

#Retirando os valores ausentes, ou seja, os exames que possuem apenas valores nulos

In [224]:
# Agrupar por 'itemid' e concatenar as unidades 'valueuom' únicas para cada 'itemid'
# Além disso, calcular a média dos valores de 'valuenum'
grouped_data = Tabela_final.groupby('itemid').agg({
    'valueuom': lambda x: ', '.join(x.dropna().unique()),
    'valuenum': 'mean'  
}).reset_index()

grouped_data.columns = ['itemid', 'valueuom_combined', 'valuenum_mean']

# Filtrar os itemid com valuenum_mean igual a zero
zero_value_items = grouped_data.query('valuenum_mean == 0')

#listando os valores que possuem média igual a zero
lista_valores = zero_value_items['itemid'].tolist()

print("Dados agrupados que possuem média igual a 0:", lista_valores)

# Filtrar os itemid onde valueuom_combined está vazio
sem_um = grouped_data[grouped_data['valueuom_combined'] == '']

# Listar os itemid com valueuom_combined vazio
lista_valores_ausentes = sem_um['itemid'].tolist()

# Exibir a lista de itemid com valueuom_combined vazio
print("Dados agrupados que não possuem unidade de medida:", lista_valores_ausentes)






Dados agrupados que possuem média igual a 0: [50800, 50812, 50827, 50828, 50854, 50855, 50857, 50871, 50873, 50874, 50876, 50879, 50880, 50887, 50919, 50920, 50932, 50933, 50937, 50939, 50940, 50941, 50942, 50943, 50944, 50946, 50948, 50955, 50975, 50979, 50981, 50999, 51017, 51071, 51073, 51074, 51075, 51076, 51079, 51086, 51090, 51091, 51092, 51098, 51103, 51134, 51137, 51145, 51147, 51150, 51151, 51152, 51154, 51155, 51156, 51158, 51159, 51164, 51167, 51168, 51177, 51178, 51182, 51183, 51184, 51188, 51191, 51192, 51193, 51197, 51198, 51211, 51213, 51216, 51217, 51219, 51230, 51233, 51234, 51235, 51236, 51238, 51239, 51243, 51246, 51247, 51252, 51260, 51262, 51264, 51266, 51267, 51268, 51278, 51287, 51290, 51292, 51294, 51296, 51299, 51303, 51305, 51306, 51307, 51309, 51310, 51313, 51314, 51315, 51317, 51319, 51320, 51321, 51323, 51324, 51325, 51326, 51328, 51331, 51332, 51333, 51334, 51335, 51336, 51337, 51338, 51339, 51340, 51341, 51342, 51373, 51388, 51398, 51399, 51400, 51402, 51

Retirando os valores/unidades de medidas ausentes e unificando as unidades de medida

In [225]:
#Retirando os itemid que possuem média igual a 0 e não tem unidade de medida, ou seja, são ausentes 
valores_para_remover = [
    "50800", "50812", "50819", "50823", "50825", "50826", "50827", "50828", 
    "50854", "50855", "50857", "50871", "50873", "50874", "50876", "50879", 
    "50880", "50887", "50919", "50920", "50932", "50933", "50939", "50940", 
    "50941", "50942", "50943", "50944", "50946", "50948", "50955", "50975", 
    "50979", "50981", "50999", "51017", "51071", "51073", "51074", "51075", 
    "51076", "51079", "51086", "51090", "51091", "51092", "51098", "51103", 
    "51134", "51137", "51145", "51147", "51150", "51151", "51152", "51154", 
    "51155", "51156", "51158", "51159", "51164", "51167", "51168", "51177", 
    "51178", "51182", "51183", "51184", "51188", "51191", "51192", "51193", 
    "51197", "51198", "51211", "51213", "51216", "51217", "51219", "51230", 
    "51233", "51234", "51235", "51236", "51238", "51239", "51243", "51246", 
    "51247", "51252", "51260", "51262", "51264", "51266", "51267", "51268", 
    "51278", "51287", "51290", "51292", "51294", "51296", "51299", "51303", 
    "51305", "51306", "51307", "51309", "51310", "51313", "51314", "51315", 
    "51317", "51319", "51320", "51321", "51323", "51324", "51325", "51326", 
    "51328", "51331", "51332", "51333", "51334", "51335", "51336", "51337", 
    "51338", "51339", "51340", "51341", "51342", "51373", "51388", "51398", 
    "51399", "51400", "51402", "51404", "51411", "51412", "51416", "51420", 
    "51423", "51424", "51425", "51426", "51460", "51462", "51463", "51464", 
    "51466", "51469", "51474", "51486", "51487", "51489", "51497", "51505", 
    "51506", "51508", "51512", "51513", "51517", "51518", "51519", "51523", 
    "51537"
]


Tabela_final['itemid'] = Tabela_final['itemid'].astype(str)


Tabela_final_2 = Tabela_final[~Tabela_final['itemid'].isin(valores_para_remover)]


# #Reutilizando a tabela grouped_data, porém retirando as colunas com média igual 0

# # Filtrar os itemid onde valueuom_combined está vazio
# sem_um = dados_agrupados[dados_agrupados['valueuom_combined'] == '']

# # Listar os itemid com valueuom_combined vazio
# lista_valores_ausentes = sem_um['itemid'].tolist()

# # Exibir a lista de itemid com valueuom_combined vazio
# print(lista_valores_ausentes)

# #Retirando os itemid que não unidade de medida, ou seja, são ausentes
# valores_para_remover_2 = ["50819", "50823", "50825", "50826", "51087", "51094", "51237", "51240"]

# dados_agrupados.loc[:, 'itemid'] = dados_agrupados['itemid'].astype(str)

# # Filtrar os dados, retirando os itemids ausentes
# dados_agrupados_2 = dados_agrupados[~dados_agrupados['itemid'].isin(valores_para_remover_2)]



Analisando quais unidades de medidas estão sendo usadas para cada exame e colocando eles na mesma unidade de medida

In [226]:
#Reutilizando a tabela dados_agrupados_2

# dados_agrupados_2.to_csv("Analisar as colunas 2.csv", index=False)

Fazendo a criação da tabela

In [227]:

Tabela_final_2 = pd.DataFrame(table.rows, columns=table.field_names)
Tabela_final_2.to_csv('Tabela final 2.csv', index=False)



Possíveis adições que seram feitas usando as tabelas microbiologyevents e drgcodes

In [228]:
# Ler DRGCODES em chunks e processar cada chunk
# for drgcodes_chunk in pd.read_csv(drgcodes_path, chunksize=chunk_size):
    
#         dados_filtrados_drgcodes.append(drgcodes_chunk)

# Ler MICROBIOLOGYEVENTS em chunks e processar cada chunk
# for microbiologyevents_chunk in pd.read_csv(microbiologyevents_path, chunksize=chunk_size):
    
#         dados_filtrados_microbiologyevents.append(microbiologyevents_chunk)

# df_filtrado_7 = pd.concat(dados_filtrados_microbiologyevents, ignore_index=True)

# df_filtrado_5 = pd.concat(dados_filtrados_drgcodes, ignore_index=True)

# df_filtrado_7 = df_filtrado_7.drop(columns=['row_id'])

# df_filtrado_5 = df_filtrado_5.drop(columns=['row_id','description','drg_severity','drg_mortality'])

#Fazendo o pivoteamento da tabela

# df_filtrado_labevents = df_filtrado_labevents.assign(value=1).pivot_table(index='subject_id', columns='itemid', values='value', fill_value=0).reset_index()

# df_filtrado_2 = df_filtrado_2.assign(value=1).pivot_table(index='subject_id', columns='religion', values='value', fill_value=0).reset_index()

# Junto_6 = pd.merge(Junto_5, df_filtrado_5, on='subject_id', how='inner')

# Tabela_final = pd.merge(Junto_6, df_filtrado_7, on='subject_id', how='inner')

# aux1 = Tabela_final.assign(value=1).pivot_table(index='subject_id', columns='item_id', values='value', fill_value=0).reset_index()


# print(df_filtrado_admissions.columns.tolist())
# print(df_filtrado_2.columns.tolist())
# print(df_filtrado_labitems.columns.tolist())
# print(Tabela_final.columns.tolist())

#PCA - scatter plot